----------
##### import Pororo

In [1]:
import pororo
from pororo import Pororo

/data/ephemeral/home/shark/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----------
##### 필요 Import

In [13]:
#import 모음

import nltk
import torch
from datasets import load_from_disk, Dataset, DatasetDict

----------
##### 초기 설정
- nltk.download 진행 시 'punkt'가 아닌 'punkt_tab'을 입력  
▶ 'punkt' 입력할 경우 mt(mt(text, src="ko", tgt="en"), tgt="en", src="ko")에서 오류 발생

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
nltk.download('punkt_tab')
mt = Pororo(task="translation", lang="multi", model='transformer.large.multi.mtpg')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


----------
##### 데이터 불러오기

In [15]:
disk = "/data/ephemeral/home/level2-mrc-nlp-12/data"

train_data = load_from_disk(disk + "/train_dataset/")
train_dataset = train_data["train"]
print(len(train_data['train']))

3952


----------
##### 역번역

In [16]:
def back_translation(text):
    return mt(mt(text, src="ko", tgt="en"), tgt="en", src="ko")

----------
##### 증강

In [17]:
def augment_data(train_dataset, limit):
    augmented_examples = []
    for idx, example in enumerate(train_dataset):
        if idx >= limit:
            break
        original_question = example['question']
        
        #역번역
        back_translated_question = back_translation(original_question)
        
        # 새로운 예제 생성
        new_example = example.copy()
        new_example['question'] = back_translated_question
        
        augmented_examples.append(new_example)
    
    # 새로운 데이터셋 생성
    aug_train_data = Dataset.from_dict(augmented_examples)
    
    return aug_train_data

In [12]:
#제한 개수
augmentation_limit = len(train_data['train'])       #전체

aug_train_data = augment_data(train_dataset, augmentation_limit)

AttributeError: 'Dataset' object has no attribute 'add_items'

----------
##### 새로운 disk에 저장

In [ ]:
combined_data = DatasetDict({'train': train_dataset + aug_train_data['train']})
combined_data.save_to_disk(disk + "/pororo_train_data/")